# Gene-depth correlations - analysis of results

This analysis, sorting gene-depth correlation results by GO term, requires two additional steps - unzipping the correlation results (in data/frem_gene_depth), and installing the goatools package (goatools 0.9.9 was used).

Install via `pip install goatools` to reproduce results.

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats import multitest
from pathlib import Path

In [2]:
# Get http://geneontology.org/ontology/go-basic.obo
# from goatools.base import download_go_basic_obo
# obo_fname = download_go_basic_obo()

In [3]:
from goatools.associations import read_ncbi_gene2go
from goatools.go_search import GoSearch

go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], namespace='BP', go2geneids=True)
# go2geneids_human.update(read_ncbi_gene2go("gene2go", taxids=[9606], namespace='MF', go2geneids=True))
# go2geneids_human.update(read_ncbi_gene2go("gene2go", taxids=[9606], namespace='CC', go2geneids=True))
print("{N} GO terms associated with human NCBI Entrez GeneIDs".format(N=len(go2geneids_human)))

srchhelp = GoSearch("go-basic.obo", go2items=go2geneids_human)

from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT

DEPRECATED read_ncbi_gene2go: USE Gene2GoReader FROM goatools.anno.genetogo_reader
DEPRECATED read_ncbi_gene2go CALLED FROM: <ipython-input-3-7b44286313cc> BY <module>
HMS:0:00:03.392002 323,107 annotations READ: gene2go 
1 taxids stored: 9606
12285 IDs in loaded association branch, BP
12285 GO terms associated with human NCBI Entrez GeneIDs
go-basic.obo: fmt(1.2) rel(2020-01-01) 47,337 GO Terms; optional_attrs(comment def relationship synonym xref)


In [4]:
def go_dict(go_id):
    gos = srchhelp.add_children_gos([go_id])
    ids = srchhelp.get_items(gos)
    return {GENEID2NT[geneid].Symbol: GENEID2NT[geneid] for geneid in ids if geneid in GENEID2NT}

def go_intersection(symbols, go_id):
    lookup = go_dict(go_id)
    return [lookup.get(symbol) for symbol in symbols if symbol in lookup]

In [5]:
pval = "p_corr"
fdr_method = 'fdr_bh'
thresh=0.01
df = pd.read_csv(Path('../data/frem_genes_depth.csv'))

df = df.dropna(subset=[pval]).set_index('gene')
df[pval] = df[pval].transform(lambda col: multitest.multipletests(col, method=fdr_method)[1])
          
sig_genes = df[df[pval]<thresh].index
len(sig_genes)

1917

Save results for ToppGene and REViGO analysis

In [6]:
# print('\n'.join(sig_genes), file=open(Path("../data/genes_depth_cca.txt", 'w')))

Load and integrate REViGO results

In [9]:
cols = [
    'term_ID',
    'description',
    'log10 p-value'
]
names = [
    'GO term ID',
    'GO term description',
    'GO term log10 p-value'
]
revigo = pd.read_csv(Path('../data/REVIGO.csv'))
df_go = revigo.head(5)[cols].rename(columns=dict(zip(cols, names))).set_index('GO term ID')

In [11]:
cols = [
#     'gene',
    'p_corr',
    'rsquared_corr',
    'r_corr'
]
names = [
#     'gene',
    'q-value (FDR-BH)',
    'Rsquared',
    'r (pearson)'
]

results = []
for go_id in df_go.index:
    genes = [gene.Symbol for gene in go_intersection(sig_genes, go_id)]
    go_genes_df = df.loc[genes].sort_values("p_corr").head(10)
    results.append(go_genes_df)
genes_df = pd.concat(results, keys=df_go.index)[cols].rename(columns=dict(zip(cols, names)))
combined = genes_df.join(df_go)
# combined.to_csv('../data/supp_depth_genes_go.csv')
combined.head(20)

q-value (FDR-BH)  Rsquared  r (pearson)  \
GO term ID gene                                                       
GO:0007268 RIT2      [2.407565619726097e-17]  0.369687     0.608019   
           KCNIP2    [1.752498173205744e-13]  0.294559    -0.542733   
           SNCG       [3.34326049318308e-12]  0.267811     0.517504   
           GRM3     [2.9268120585820327e-09]  0.202726     0.450251   
           HTR1F    [4.2709826081279284e-08]  0.175566     0.419006   
           KCNQ3     [6.665067835502736e-08]  0.170981     0.413498   
           NPTX2     [9.265222945309073e-08]  0.167573    -0.409357   
           GRID2     [9.335867758879824e-07]  0.143347    -0.378612   
           CACNA1E  [2.5303035028424627e-06]  0.132721    -0.364309   
           EXOC4     [8.101670936277667e-06]  0.120197    -0.346695   
GO:0048812 NTNG1     [5.020572035815342e-11]  0.242395     0.492336   
           NEFL      [5.797405043042319e-06]  0.123812     0.351869   
           EPB41L3   [1.987023608739026e-05]  0.110458    -0.332352   
           TNIK      [2.905425011480746e-05]  0.106313     0.326056   
           BCL2      [3.479349204621642e-05]  0.104342    -0.323020   
           PHACTR1   [4.948108620246109e-05]  0.100485    -0.316993   
           PAX2     [0.00013047558211803491]  0.089819    -0.299699   
           PARD3    [0.00013059676627305748]  0.089809    -0.299682   
           CCK      [0.00018984490863934366]  0.085679     0.292709   
           DCC      [0.00026007573649947975]  0.082197     0.286701   

                                GO term description  GO term log10 p-value  
GO term ID gene                                                             
GO:0007268 RIT2      chemical synaptic transmission               -12.5035  
           KCNIP2    chemical synaptic transmission               -12.5035  
           SNCG      chemical synaptic transmission               -12.5035  
           GRM3      chemical synaptic transmission               -12.5035  
           HTR1F     chemical synaptic transmission               -12.5035  
           KCNQ3     chemical synaptic transmission               -12.5035  
           NPTX2     chemical synaptic transmission               -12.5035  
           GRID2     chemical synaptic transmission               -12.5035  
           CACNA1E   chemical synaptic transmission               -12.5035  
           EXOC4     chemical synaptic transmission               -12.5035  
GO:0048812 NTNG1    neuron projection morphogenesis               -11.1229  
           NEFL     neuron projection morphogenesis               -11.1229  
           EPB41L3  neuron projection morphogenesis               -11.1229  
           TNIK     neuron projection morphogenesis               -11.1229  
           BCL2     neuron projection morphogenesis               -11.1229  
           PHACTR1  neuron projection morphogenesis               -11.1229  
           PAX2     neuron projection morphogenesis               -11.1229  
           PARD3    neuron projection morphogenesis               -11.1229  
           CCK      neuron projection morphogenesis               -11.1229  
           DCC      neuron projection morphogenesis               -11.1229